# Project Title
### Data Engineering Capstone Project

#### Project Summary

The project follows the follow steps:
* Step 1: Scope the Project
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
from pyspark.sql import SparkSession
import configparser
from datetime import datetime,timedelta
import pyspark.sql.functions as f
from pyspark.sql.types import *
import psycopg2

## Step 1: Scope the Project



### I will use four data set provided by udacity to define fact and dimension table

### Data Sets
* **I94 Immigration Data**: This data comes from the US National Tourism and Trade Office.
* **World Temperature Data**: This dataset came from Kaggle.
* **U.S. City Demographic Data**: This data comes from OpenSoft.
* **Airport Code Table**: This is a simple table of airport codes and corresponding cities

## Step 2: Explore and Assess the Data

### Explore the data
Identify data quality issues, like missing values, duplicate data, etc.

### I. I94 Immigration Data

In [2]:
# Read in the data here
immigration_path ='../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immigration = pd.read_sas(immigration_path, 'sas7bdat', encoding="ISO-8859-1")

In [3]:
df_immigration.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [4]:
df_immigration.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [5]:
# count data
df_immigration.shape

(3096313, 28)

### II. World Temperature Data

In [6]:
temperature_path = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature = pd.read_csv(temperature_path)

In [7]:
df_temperature.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [8]:
df_temperature.columns

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')

In [9]:
# count data temperature
df_temperature.shape

(8599212, 7)

### III. U.S. City Demographic Data

In [10]:
df_demographic = pd.read_csv("us-cities-demographics.csv", delimiter = ";")

In [11]:
df_demographic.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [12]:
df_demographic.columns

Index(['City', 'State', 'Median Age', 'Male Population', 'Female Population',
       'Total Population', 'Number of Veterans', 'Foreign-born',
       'Average Household Size', 'State Code', 'Race', 'Count'],
      dtype='object')

In [13]:
# count data
df_demographic.shape

(2891, 12)

### IV. Airport Code Table

In [14]:
df_airport = pd.read_csv('airport-codes_csv.csv')

In [15]:
df_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [16]:
df_airport.columns

Index(['ident', 'type', 'name', 'elevation_ft', 'continent', 'iso_country',
       'iso_region', 'municipality', 'gps_code', 'iata_code', 'local_code',
       'coordinates'],
      dtype='object')

In [17]:
# count data
df_airport.shape

(55075, 12)

### Check null and duplicate data

**Check Null**

In [18]:
# Immigration
check_null_immi = df_immigration.isnull().sum()
pd.DataFrame(check_null_immi)

,0
cicid,0
i94yr,0
i94mon,0
i94cit,0
i94res,0
i94port,0
arrdate,0
i94mode,239
i94addr,152372
depdate,142457


In [19]:
# Temperature
check_null_temperature = df_temperature.isnull().sum()
pd.DataFrame(check_null_temperature)

,0
dt,0
AverageTemperature,364130
AverageTemperatureUncertainty,364130
City,0
Country,0
Latitude,0
Longitude,0


In [20]:
# Demographic
check_null_demographic = df_demographic.isnull().sum()
pd.DataFrame(check_null_demographic)

,0
City,0
State,0
Median Age,0
Male Population,3
Female Population,3
Total Population,0
Number of Veterans,13
Foreign-born,13
Average Household Size,16
State Code,0


In [21]:
# Airport
check_null_airport = df_airport.isnull().sum()
pd.DataFrame(check_null_airport)

,0
ident,0
type,0
name,0
elevation_ft,7006
continent,27719
iso_country,247
iso_region,0
municipality,5676
gps_code,14045
iata_code,45886


### Check duplicate

**Check duplicate if have record duplicate.**

**In Cleaning Steps use df = df.drop_duplicates()**

In [23]:
# If have record duplicate, In Cleaning Steps
# use df = df.drop_duplicates()
print("Check duplicate:")
df_check = {
    "Immigration": df_immigration,
    "Temperature": df_temperature,
    "Demographic": df_demographic,
    "Airport": df_airport
}

for name, df in df_check.items():
    if df.shape[0] == df.drop_duplicates().shape[0]:
        print(f"{name} haven't record duplicated!.")
    else:
        print(f"Data {name} have record duplicated!.")

Immigration haven't record duplicated!.
Temperature haven't record duplicated!.
Demographic haven't record duplicated!.
Airport haven't record duplicated!.


### Read file I94_SAS_Labels_Descriptions.SAS
File contains values: code and value description for:
1. country example  236 =  'AFGHANISTAN'
2. airport example 'ANC'	=	'ANCHORAGE, AK
3. stage example 'FL'='FLORIDA'
4. mode example 2 = 'Sea'.

I94VISA - Visa codes collapsed into three categories:
   * 1 = Business
   * 2 = Pleasure
   * 3 = Student
   
Use define dimention table

In [3]:
with open("I94_SAS_Labels_Descriptions.SAS") as f:
    contents = f.readlines()

In [27]:
# Country
# 10:274 is range valid values country
country_data = {}
for countries in contents[10:274]:
    pair = countries.split('=')
    country_code = pair[0].strip()
    country_name = pair[1].strip().strip("'")
    country_data[country_code] = country_name

In [28]:
df_country_sas = pd.DataFrame(list(country_data.items()), columns = ["code", "country"])
df_country_sas.head()

,code,country
0,236,AFGHANISTAN
1,101,ALBANIA
2,316,ALGERIA
3,102,ANDORRA
4,324,ANGOLA


In [27]:
# Airport
# 302:894 is range valid for airport
airport_data = {}
for airports in contents[302:894]:
    pair = airports.split('=')
    airport_code = pair[0].strip().strip("\t").strip("'")
    airport_name = pair[1].strip().strip("'")
    airport_data[airport_code] = airport_name

In [28]:
df_airport_sas = pd.DataFrame(list(airport_data.items()), columns = ["code", "airport"])
df_airport_sas.head()

,code,airport
0,ALC,"ALCAN, AK"
1,ANC,"ANCHORAGE, AK"
2,BAR,"BAKER AAF - BAKER ISLAND, AK"
3,DAC,"DALTONS CACHE, AK"
4,PIZ,"DEW STATION PT LAY DEW, AK"


In [29]:
# Stage
# 982:1036 is range valid for stage
stage_data = {}
for stages in contents[982:1036]:
    pair = stages.split('=')
    stage_code = pair[0].strip().strip("\t").strip("'")
    stage_name = pair[1].strip().strip("'")
    stage_data[stage_code] = stage_name

In [71]:
df_stage_sas = pd.DataFrame(list(stage_data.items()), columns = ["code", "stage"])
df_stage_sas.head()

,code,stage
0,AK,ALASKA
1,AZ,ARIZONA
2,AR,ARKANSAS
3,CA,CALIFORNIA
4,CO,COLORADO


In [4]:
# Mode
# 972:975 is range valid for mode
mode_data = {}
for citys in contents[972:975]:
    pair = citys.split('=')
    mode_code = pair[0].strip().strip("\t").strip("'")
    mode_name = pair[1].strip().strip("'")
    mode_data[mode_code] = mode_name

In [5]:
df_mode_sas = pd.DataFrame(list(mode_data.items()), columns = ["code", "mode"])
df_mode_sas.head()

,code,mode
0,1,Air
1,2,Sea
2,3,Land


### Cleaning Steps
Document steps necessary to clean the data

### 1. Define fact and dimension table
### 2. Run data with spark

**1. Define fact and dimension table**

Filter columns contains value null large and define in file **I94_SAS_Labels_Descriptions**, will not use columns in fact


Define name columns for easy understanding

Since there are no duplicates, there is no need to reset df

**DTADFILE** - Character Date Field - Date added to I-94 Files - CIC does not use */


**VISAPOST** - Department of State where where Visa was issued - CIC does not use */


**OCCUP** - Occupation that will be performed in U.S. - CIC does not use */


**ENTDEPA** - Arrival Flag - admitted or paroled into the U.S. - CIC does not use */

### Immigration

In [33]:
# Immigration fact
# Eliminate columns
# visapost 1881250
# occup 3088187
# entdepu 3095921
# insnum 2982605
# Define name columns for easy understanding
pd.set_option("display.max_columns", None)
fact_immigration = df_immigration[['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
                                   'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count', 'entdepd',
                                   'matflag', 'biryear', 'dtaddto', 'gender', 'airline',
                                   'admnum', 'fltno', 'visatype']]

fact_immigration.columns = ['cic_id', 'year', 'month', 'cit', 'res', 'port', 'arr_date', 
                            'mode', 'address', 'dep_date', 'birth_day', 'visa', 'total',
                            'entdepd', 'matflag', 'birth_year','dtaddto','gender', 'airline',
                            'admnum', 'fltno', 'visatype'
                           ]

fact_immigration.head()

,cic_id,year,month,cit,res,port,arr_date,mode,address,dep_date,birth_day,visa,total,entdepd,matflag,birth_year,dtaddto,gender,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,1.0,NaN,NaN,1979.0,10282016,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,NaN,NaN,1991.0,D/S,M,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,O,M,1961.0,09302016,M,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,O,M,1988.0,09302016,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,O,M,2012.0,09302016,NaN,AA,9.246846e+10,00199,B2


In [34]:
fact_immigration.columns

Index(['cic_id', 'year', 'month', 'cit', 'res', 'port', 'arr_date', 'mode',
       'address', 'dep_date', 'birth_day', 'visa', 'total', 'entdepd',
       'matflag', 'birth_year', 'dtaddto', 'gender', 'airline', 'admnum',
       'fltno', 'visatype'],
      dtype='object')

### Immigration dimension
In here i will split two immigration dimension from fact
1. dimension_personal
2. dimension_airline

In [35]:
# dimension_personal
dimension_personal = fact_immigration[['cic_id', 'year', 'month', 'address',
                                       'birth_day','birth_year', 'gender']]

dimension_personal.head()

,cic_id,year,month,address,birth_day,birth_year,gender
0,6.0,2016.0,4.0,NaN,37.0,1979.0,NaN
1,7.0,2016.0,4.0,AL,25.0,1991.0,M
2,15.0,2016.0,4.0,MI,55.0,1961.0,M
3,16.0,2016.0,4.0,MA,28.0,1988.0,NaN
4,17.0,2016.0,4.0,MA,4.0,2012.0,NaN


In [36]:
# dimension_airline
dimension_airline = fact_immigration[['cic_id', 'port', 'airline', 'visa',
                                      'visatype', 'dep_date'
                                     ]]
dimension_airline.head()


,cic_id,port,airline,visa,visatype,dep_date
0,6.0,XXX,NaN,2.0,B2,NaN
1,7.0,ATL,NaN,3.0,F1,NaN
2,15.0,WAS,OS,2.0,B2,20691.0
3,16.0,NYC,AA,2.0,B2,20567.0
4,17.0,NYC,AA,2.0,B2,20567.0


### Temperature fact

In [37]:
df_temperature.columns

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')

In [38]:
fact_temperature = df_temperature[['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 
                                   'City','Country', 'Latitude', 'Longitude']]

fact_temperature.columns = ['dt', 'avg_temperature', 'avg_uncertainty', 'city', 'country',
                            'latitude', 'longitude'
                           ]
fact_temperature.head()                         

,dt,avg_temperature,avg_uncertainty,city,country,latitude,longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


### Demoraphic fact

In [39]:
df_demographic.columns

Index(['City', 'State', 'Median Age', 'Male Population', 'Female Population',
       'Total Population', 'Number of Veterans', 'Foreign-born',
       'Average Household Size', 'State Code', 'Race', 'Count'],
      dtype='object')

In [40]:
fact_demographic = df_demographic[['City', 'State', 'Median Age', 'Male Population', 'Female Population',
       'Total Population', 'Number of Veterans', 'Foreign-born',
       'Average Household Size', 'State Code', 'Race', 'Count']]
fact_demographic.columns = ['city', 'state', 'median_age', 'male_population', 'female_population',
                            'total_population', 'number_of_veterans', 'foreign_born',
                            'avg_house_hold_size', 'stage_code', 'race', 'total'
                           ]
fact_demographic.head()

,city,state,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,avg_house_hold_size,stage_code,race,total
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [41]:
fact_demographic.columns

Index(['city', 'state', 'median_age', 'male_population', 'female_population',
       'total_population', 'number_of_veterans', 'foreign_born',
       'avg_house_hold_size', 'stage_code', 'race', 'total'],
      dtype='object')

**Dimension demographic**

In [42]:
dimension_population = fact_demographic[['city', 'state', 'male_population', 'female_population',
                                         'number_of_veterans', 'foreign_born', 'race']]
dimension_population.head()

,city,state,male_population,female_population,number_of_veterans,foreign_born,race
0,Silver Spring,Maryland,40601.0,41862.0,1562.0,30908.0,Hispanic or Latino
1,Quincy,Massachusetts,44129.0,49500.0,4147.0,32935.0,White
2,Hoover,Alabama,38040.0,46799.0,4819.0,8229.0,Asian
3,Rancho Cucamonga,California,88127.0,87105.0,5821.0,33878.0,Black or African-American
4,Newark,New Jersey,138040.0,143873.0,5829.0,86253.0,White


In [72]:
dimention_city_statistics = fact_demographic[['city', 'state', 'stage_code','total_population', 'avg_house_hold_size']]
dimention_city_statistics.head()

,city,state,stage_code,total_population,avg_house_hold_size
0,Silver Spring,Maryland,MD,82463,2.60
1,Quincy,Massachusetts,MA,93629,2.39
2,Hoover,Alabama,AL,84839,2.58
3,Rancho Cucamonga,California,CA,175232,3.18
4,Newark,New Jersey,NJ,281913,2.73


### Airport fact

In [48]:
fact_airport = df_airport
fact_airport.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### Read data with spark

In [7]:
config = configparser.ConfigParser()
config.read_file(open('aws.cfg'))

os.environ["AWS_ACCESS_KEY_ID"] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"] = config['AWS']['AWS_SECRET_ACCESS_KEY']
output_data = config['AWS']['OUTPUT_DATA']

In [8]:
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0").\
enableHiveSupport().getOrCreate()


In [50]:
# Country in file I94_SAS_Labels_Descriptions
dimention_country = spark.createDataFrame(df_country_sas)
dimention_country.printSchema()

root
 |-- code: string (nullable = true)
 |-- country: string (nullable = true)



In [51]:
# Airport in file I94_SAS_Labels_Descriptions
dimention_airport = spark.createDataFrame(df_airport_sas)
dimention_airport.printSchema()

root
 |-- code: string (nullable = true)
 |-- airport: string (nullable = true)



In [68]:
# Stage in file I94_SAS_Labels_Descriptions
dimention_stage = spark.createDataFrame(df_stage_sas)
dimention_stage.printSchema()

root
 |-- code: string (nullable = true)
 |-- city: string (nullable = true)



In [9]:
# Mode in file I94_SAS_Labels_Descriptions
dimention_mode = spark.createDataFrame(df_mode_sas)
dimention_mode.printSchema()

root
 |-- code: string (nullable = true)
 |-- mode: string (nullable = true)



**Because converting DataFrame from pandas to spark have issue**

**Error: can not merge type**

**I will continute process in step 4 "Step 4: Run Pipelines to Model the Data"**

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

### Table
1. fact_immigration
2. dimension_personal
3. dimension_airline
---
4. fact_temperature
---
5. fact_demographic
6. dimension_population
7. dimention_city_statistics
----
8. fact_airport
---
From file **I94_SAS_Labels_Descriptions.SAS**.
9. dimension_stage
10. dimension_country
11. dimention_mode
12. dimention_airport

**Diagram Schema**

![](./img/diagram.png)

**Why you chose that model**
1. Split into sub-tables for easy querying later 1 - 1.
* Example:
* fact_immigration 1 - 1 dimension_personal
* fact_immigration 1 - 1 dimension_airline
* When the user only wants to get the list of people in the immagration, instead of having to query the whole table, the user only needs to query at dimension_personal.
2. See 1-n relationships between tables.
* Example:
* dimention_mode 1 - n fact_immigration
* Get name mode columns in fact_immigration

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

### Run data with spark and store in S3
### Define fact and dimension table

In [11]:
# immigration_path = "../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat"
# fact_immigrations = spark.read.format('com.github.saurfang.sas.spark').load(immigration_path)
# Read file from folder sas_data data <=> "../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat"
fact_immigrations = spark.read.parquet("sas_data")

In [9]:
fact_immigrations.count()

3096313

In [10]:
fact_immigrations.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [12]:
fact_immigrations.createOrReplaceTempView("fact_immigration")
fact_immigration = spark.sql("""
    SELECT
        cicid as cic_id,
        i94yr as year,
        i94mon as month,
        i94cit as cit,
        i94res as res,
        i94port as port,
        arrdate as arr_date,
        i94mode as mode,
        i94addr as address,
        depdate as dep_date,
        i94bir as birth_day,
        i94visa as visa,
        count as total,
        entdepd,
        matflag,
        biryear as birth_year,
        dtaddto,
        gender,
        airline,
        admnum,
        fltno,
        visatype
    FROM fact_immigration
""")

In [12]:
fact_immigration.printSchema()

root
 |-- cic_id: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- cit: double (nullable = true)
 |-- res: double (nullable = true)
 |-- port: string (nullable = true)
 |-- arr_date: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- address: string (nullable = true)
 |-- dep_date: double (nullable = true)
 |-- birth_day: double (nullable = true)
 |-- visa: double (nullable = true)
 |-- total: double (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- birth_year: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [13]:
dimension_personal = fact_immigration.select('cic_id', 'year', 'month', 'address', 'birth_day',
                                             'birth_year', 'gender'
                                            )
dimension_personal.printSchema()

root
 |-- cic_id: double (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- address: string (nullable = true)
 |-- birth_day: double (nullable = true)
 |-- birth_year: double (nullable = true)
 |-- gender: string (nullable = true)



### Run etl.py
### Read file -> write file store in S3

![](./img/evd.png)

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

### Check Data has store in S3 and count data

### Refer DataQuality.ipynb

![](./img/evd2.png)

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

![](./img/evd3.png)

### Refer description_columns.txt

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

### Tools and Technologies
1. AWS S3 for data storage
2. Pandas for sample data set exploratory data analysis
3. PySpark for large data set data processing to transform staging table to dimensional table

### Propose how often the data should be updated and why.
1. All tables should be update in an append-only mode.
2. Airport should be updated daily, because of the high frequency
3. The temperature should be updated every year, as it doesn't change much

### Write a description of how you would approach the problem differently under the following scenarios:

**The data was increased by 100x.**

If Spark with standalone server mode can not process 100x data set, we could consider to put data in AWS EMR which is a distributed data cluster for processing large data sets on cloud

**The data populates a dashboard that must be updated on a daily basis by 7am every day.**

Apache Airflow could be used for building up a ETL data pipeline to regularly update the date and populate a report. Apache Airflow also integrate with Python and AWS very well. More applications can be combined together to deliever more powerful task automation.

**The database needed to be accessed by 100+ people.**

AWS Redshift can handle up to 500 connections. If this SSOT database will be accessed by 100+ people, we can move this database to Redshift with confidence to handle this request. Cost/Benefit analysis will be needed if we are going be implement this cloud solution.